In [1]:
import molpert as mpt
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem.Descriptors import CalcMolDescriptors
from rdkit import RDLogger
from rdkit.Chem import AllChem, DataStructs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from PIL import Image
import textwrap
from tqdm import tqdm

import sascorer

from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina

# Suppress RDKit warnings and errors
RDLogger.DisableLog('rdApp.*')

from vina import Vina
import subprocess
from pathlib import Path

def propagate_chemical_space_trajectory(
    starting_smiles,            # smiles of the starting structure
    iteration,         # how many times to do the perturbation for, including the first structure
    SA_score_threshold # only get structures with SA score lower than the threshold 
):
    def sensible_structure(molecule):
    # Only accept structures that raise no flags when using RDKit sanitization
        try:
            result = Chem.SanitizeMol(molecule)
            return result == Chem.rdmolops.SanitizeFlags.SANITIZE_NONE
        except Exception as e:
            # You can log or inspect e if needed
            return False

    def SA_score(molecule):
    # Only accept structures with SA score under some threshold
    # The threshold value is kinda arbitrary (and would depend on the system)
        return sascorer.calculateScore(molecule) < SA_score_threshold
    
    # Molecule array
    molecule = Chem.MolFromSmiles(starting_smiles, sanitize=True)
    mol_array = [Chem.Mol(molecule)]

    # Set constraint to only get 'sensible' structures
    # Could add pains filter here 
    constraints = mpt.MolecularConstraints()
    constraints.SetMoleculeConstraint(
        molecule_constraint=sensible_structure
    )
    constraints.SetMoleculeConstraint(
        molecule_constraint=SA_score
    )

    # Iterate for a given number of steps 
    for i in tqdm(range(iteration-1)):
        # Build molecule on rdkit & tag them which is needed for molpert
        mpt.TagMolecule(molecule)
        
        # Perturber and random number
        perturber = mpt.MoleculePerturber()
        #perturber = mpt.MoleculePerturber(use_aromatic_bonds=False, acyclic_can_be_aromatic=False)
            # Idk what the best setting for these / especially how to deal with aromaticity 
        prng = mpt.PRNG()

        # Actually perform perturbation 
        perturbation = perturber.Perturbation(
            molecule=molecule,
            prng=prng,
            constraints=constraints
        )
        mpt.ExecutePerturbation(perturbation, molecule)

        # Sanitize
        Chem.SanitizeMol(molecule)
        
        # Add the new perturbed molecule to the array
        # Reset smiles_i to be used for next perturbation 
        mol_array.append(Chem.Mol(molecule))

    return np.array(mol_array)

def mol_movie(mols, outfile, fps=6, img_size=200):
    """
    Create an animated GIF from a list of RDKit Mol objects,
    aligning each to the previous one based on their common substructure.
    
    mols: list of RDKit Mol objects
    outfile: name of the output GIF
    fps: frames per second
    """
    # Ensure molecules have coordinates for drawing
    for mol in mols:
        AllChem.Compute2DCoords(mol)

    images = []

    for i, mol in tqdm(enumerate(mols)):
        if i > 0:
            # Find maximum common substructure
            core = Chem.MolFromSmarts(Chem.MolToSmarts(mols[i-1]))
            match1 = mols[i-1].GetSubstructMatch(core)
            match2 = mol.GetSubstructMatch(core)

            if match1 and match2:
                AllChem.AlignMol(mol, mols[i-1], atomMap=list(zip(match2, match1)))

        # Draw molecule
        img = Draw.MolToImage(mol, size=(img_size, img_size))
    
        # Add index text
        draw = ImageDraw.Draw(img)
        draw.text((10, 10), str(i), font=ImageFont.load_default(), fill=(0, 0, 0))

        images.append(img)
        
        #img = Draw.MolToImage(mol, size=(300, 300))
        #images.append(img)

    # Save as animated GIF using PIL
    images[0].save(
        f"{outfile}.gif",
        save_all=True,
        append_images=images[1:],
        duration=int(1000/fps),  # milliseconds per frame
        loop=0
    )
    print(f"Movie saved to {outfile}.gif")

def mol_similarity(
    mol1, 
    mol2
):
    # Generate Morgan fingerprints (ECFP4 radius=2)
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, radius=2, nBits=2048)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, radius=2, nBits=2048)

    return DataStructs.TanimotoSimilarity(fp1, fp2)

def trajectory_quantification(
    mol_array
):
# For each trajectory, want to calculate 3 sets of quantities
# a) graph quantities: # of atoms, # of bonds, topological complexity 
# https://www.rdkit.org/docs/source/rdkit.Chem.GraphDescriptors.html
# b) Lipinski quantities: Hbond donor, Hbond acceptor, molecular mass, ClogP
# c) Fingerprint similiariy wrt starting point
# d) SA score
    rows = []

    for mol in mol_array:
        rows.append({
            # a) Graph quantities
            "NumAtoms": mol.GetNumAtoms(),
            "NumBonds": mol.GetNumBonds(),
            "BertzCT": Chem.GraphDescriptors.BertzCT(mol),
            # b) Lipinski quantities
            "HBA": Chem.Lipinski.NumHAcceptors(mol),
            "HBD": Chem.Lipinski.NumHDonors(mol),
            "MolWt": Chem.Descriptors.MolWt(mol),
            "ClogP": Chem.Descriptors.MolLogP(mol),
            # c) Fingerprint similarity w.r.t. starting point
            "FingerprintSim": mol_similarity(mol, mol_array[0]),
            # d) SA score
            "SA_Score": sascorer.calculateScore(mol),
            # Extra
            "SMILES": Chem.MolToSmiles(mol)
        })
    
    # Create DataFrame with explicit column order
    df = pd.DataFrame(rows, columns=[
        "NumAtoms", "NumBonds", "BertzCT",
        "HBA", "HBD", "MolWt", "ClogP",
        "FingerprintSim", "SA_Score", "SMILES"
    ])

    return df    

def plot_quantification_traj(
    df,
    show_initial_value
):
    fig, axs = plt.subplots(2, 3, figsize=(12, 6))  # 1 row, 3 columns
    font = 14
    
    axs[0,0].plot(df['BertzCT'])
    axs[0,0].tick_params(axis='both', labelsize=font)
    axs[0,0].set_xlabel('Perturbations',size=font);
    axs[0,0].legend(['Topological Index'],fontsize=font-4);

    axs[0,1].plot(df['HBA'])
    axs[0,1].plot(df['HBD'])
    axs[0,1].tick_params(axis='both', labelsize=font)
    axs[0,1].legend(['# HBA','# HBD'],fontsize=font-4);
    axs[0,1].set_xlabel('Perturbations',size=font);

    axs[0,2].plot(df['MolWt'])
    axs[0,2].tick_params(axis='both', labelsize=font)
    axs[0,2].set_xlabel('Perturbations',size=font);
    axs[0,2].legend(['Molecular Weight'],fontsize=font-4);

    axs[1,0].plot(df['ClogP'])
    axs[1,0].tick_params(axis='both', labelsize=font)
    axs[1,0].set_xlabel('Perturbations',size=font);
    axs[1,0].legend(['Partition Coefficient'],fontsize=font-4);

    axs[1,1].plot(df['FingerprintSim'])
    axs[1,1].tick_params(axis='both', labelsize=font)
    axs[1,1].set_xlabel('Perturbations',size=font);
    axs[1,1].legend(['Fingerprint Similarity\nwrt Starting Point'],fontsize=font-4);

    axs[1,2].plot(df['SA_Score'])
    axs[1,2].tick_params(axis='both', labelsize=font)
    axs[1,2].set_xlabel('Perturbations',size=font);
    axs[1,2].legend(['SA Score'],fontsize=font-4);

    if show_initial_value:
        axs[0,0].axhline(df.iloc[0]['BertzCT'], color='red', linestyle=':')
        axs[0,1].axhline(df.iloc[0]['HBA'], color='tab:blue', linestyle=':')
        axs[0,1].axhline(df.iloc[0]['HBD'], color='tab:orange', linestyle=':')
        axs[0,2].axhline(df.iloc[0]['MolWt'], color='red', linestyle=':')
        axs[1,0].axhline(df.iloc[0]['ClogP'], color='red', linestyle=':')
        axs[1,1].axhline(df.iloc[0]['FingerprintSim'], color='red', linestyle=':')
        axs[1,2].axhline(df.iloc[0]['SA_Score'], color='red', linestyle=':')
    
    plt.tight_layout()
    plt.show()

def plot_quantification_hist(
    df,
    show_initial_value
):
    fig, axs = plt.subplots(2, 3, figsize=(12, 6))  # 1 row, 3 columns
    font = 14
    
    axs[0,0].hist(df['BertzCT'],bins=20,density=True)
    axs[0,0].tick_params(axis='both', labelsize=font)
    axs[0,0].legend(['Topological Index'],fontsize=font-4);

    axs[0,1].hist(df['HBA'],bins=20,density=True)
    axs[0,1].hist(df['HBD'],bins=20,density=True)
    axs[0,1].tick_params(axis='both', labelsize=font)
    axs[0,1].legend(['# HBA','# HBD'],fontsize=font-4);

    axs[0,2].hist(df['MolWt'],bins=20,density=True)
    axs[0,2].tick_params(axis='both', labelsize=font)
    axs[0,2].legend(['Molecular Weight'],fontsize=font-4);

    axs[1,0].hist(df['ClogP'],bins=20,density=True)
    axs[1,0].tick_params(axis='both', labelsize=font)
    axs[1,0].legend(['Partition Coefficient'],fontsize=font-4);

    axs[1,1].hist(df['FingerprintSim'],bins=20,density=True)
    axs[1,1].tick_params(axis='both', labelsize=font)
    axs[1,1].legend(['Fingerprint Similarity\nwrt Starting Point'],fontsize=font-4);

    axs[1,2].hist(df['SA_Score'],bins=20,density=True)
    axs[1,2].tick_params(axis='both', labelsize=font)
    axs[1,2].legend(['SA Score'],fontsize=font-4);
    
    if show_initial_value:
        axs[0,0].axvline(df.iloc[0]['BertzCT'], color='red', linestyle=':')
        axs[0,1].axvline(df.iloc[0]['HBA'], color='tab:blue', linestyle=':')
        axs[0,1].axvline(df.iloc[0]['HBD'], color='tab:orange', linestyle=':')
        axs[0,2].axvline(df.iloc[0]['MolWt'], color='red', linestyle=':')
        axs[1,0].axvline(df.iloc[0]['ClogP'], color='red', linestyle=':')
        axs[1,1].axvline(df.iloc[0]['FingerprintSim'], color='red', linestyle=':')
        axs[1,2].axvline(df.iloc[0]['SA_Score'], color='red', linestyle=':')
    
    plt.tight_layout()
    plt.show()

def tanimoto_distance_matrix(fp_list):
    """Calculate distance matrix for fingerprint list"""
    dissimilarity_matrix = []
    # Notice how we are deliberately skipping the first and last items in the list
    # because we don't need to compare them against themselves
    for i in range(1, len(fp_list)):
        # Compare the current fingerprint against all the previous ones in the list
        similarities = DataStructs.BulkTanimotoSimilarity(fp_list[i], fp_list[:i])
        # Since we need a distance matrix, calculate 1-x for every element in similarity matrix
        dissimilarity_matrix.extend([1 - x for x in similarities])
    return dissimilarity_matrix

def cluster_fingerprints(fingerprints, cutoff=0.2):
    """Cluster fingerprints
    Parameters:
        fingerprints
        cutoff: threshold for the clustering
    """
    # Calculate Tanimoto distance matrix
    distance_matrix = tanimoto_distance_matrix(fingerprints)
    # Now cluster the data with the implemented Butina algorithm:
    clusters = Butina.ClusterData(distance_matrix, len(fingerprints), cutoff, isDistData=True)
    clusters = sorted(clusters, key=len, reverse=True)
    return clusters

def get_cluster_medoids(
    fingerprints, 
    clusters,
    cluster_size_threshold # Only save info for cluster with certain amount of elements innit
):
    medoids = []
    for cluster in clusters:
        if len(cluster) < cluster_size_threshold:
            continue
        else:
            # Compute average similarity to all others in the cluster
            avg_sims = []
            for idx in cluster:
                sims = DataStructs.BulkTanimotoSimilarity(fingerprints[idx], [fingerprints[i] for i in cluster if i != idx])
                avg_sims.append(np.mean(sims))
            avg_sims = np.array(avg_sims)
    
            # Select the index with the highest average similarity
            medoid_idx = cluster[np.argmax(avg_sims)]
            medoids.append(medoid_idx)
    return np.array(medoids)

def cluster_trajectory(
    mol_array, 
    cluster_size_threshold
):
    # https://projects.volkamerlab.org/teachopencadd/talktorials/T005_compound_clustering.html
    # use case would be: 
    # traj_concat = np.concatenate(trajectories)           // concatenate trajectories which most likely will be in replicates
    # clusters, medoids = cluster_trajectory(traj_concat)  // get cluster and medoid info (in index of traj_concat) 
    # then traj_concat[medoids] would be the representative structures that will be docked (or sth) 
    
    # Create fingerprints for all molecules
    rdkit_gen = rdFingerprintGenerator.GetRDKitFPGenerator(maxPath=5)
    fingerprints = [rdkit_gen.GetFingerprint(mol) for mol in mol_array]

    # Cluster based on fingerprints
    clusters = cluster_fingerprints(fingerprints)
    medoids = get_cluster_medoids(fingerprints, clusters, cluster_size_threshold)

    return clusters, medoids

def prepare_ligand(
    lig_name,
    lig_smiles,
    prepare_ligand_script_path
):
    try:
        subprocess.run(
            ["sh", prepare_ligand_script_path, lig_name, lig_smiles],
            check=True
        )
    except subprocess.CalledProcessError as e:
        print(f"Skipping {lig_name} due to error: {e}")

def prepare_receptor(
    receptor_name,
    receptor_pdb_path,
    prepare_receptor_script_path
):
    subprocess.run(
    ["sh", prepare_receptor_script_path, receptor_name, receptor_pdb_path],
    check=True
    );

def vina_setup_receptor(
    receptor_pdbqt,
    grid_center,
):
# https://github.com/ccsb-scripps/AutoDock-Vina/tree/develop
# https://github.com/janash/iqb-2024/blob/main/docking_single_ligand.ipynb
# https://autodock-vina.readthedocs.io/en/latest/docking_python.html
    v = Vina(sf_name='vina')
    v.set_receptor(rigid_pdbqt_filename=receptor_pdbqt)
    v.compute_vina_maps(center=grid_center, box_size=[20, 20, 20])
    return v 

def vina_dock_ligand(
    v, # vina model
    ligand_pdbqt,
    output_folder,
    output_name#, exhaustiveness=100
):
# https://github.com/ccsb-scripps/AutoDock-Vina/tree/develop
# https://github.com/janash/iqb-2024/blob/main/docking_single_ligand.ipynb
# https://autodock-vina.readthedocs.io/en/latest/docking_python.html
    v.set_ligand_from_file(ligand_pdbqt)    
    # Dock the ligand
    v.dock()#exhaustiveness=exhaustiveness)
    v.optimize()
    v.write_poses(output_folder+output_name, n_poses=5, overwrite=True)
    return v

In [2]:
def propagate_chemical_space_trajectory_reps(
    starting_point,
    n_rep,
    n_pert,
    SA_threshold
):
    trajs = []
    for i in range(n_rep):
        trajs.append(
            propagate_chemical_space_trajectory(
                starting_point,
                n_pert,
                SA_threshold)
        )
    return trajs

def prepare_ligand_batch(
    medoids,
    trajs_concat,
    epoch,
    prepare_ligand_script_path
):
    for idx in medoids:
        lig_name   = f"epoch_{epoch}_lig_{idx}"
        lig_smiles = Chem.MolToSmiles(trajs_concat[idx])
        prepare_ligand(
            lig_name,
            lig_smiles,
            prepare_ligand_script_path
        )
    # Move ligands that failed preparation step to a folder
    subprocess.run("mv *.sdf failed_ligands", shell=True);

def vina_dock_ligand_batch(
    prepared_ligands_path,
    epoch,
    v_receptor,
    output_folder
):
    # Dock ligands to prepared receptor grid 
    docking_result = []
    for i, pdbqt_file in tqdm(enumerate(Path(prepared_ligands_path).glob(f"epoch_{epoch}_*.pdbqt"))):
        lig_name = pdbqt_file.name.split(".")[0]
        print(f"Docking: {lig_name}")      
        result = vina_dock_ligand(
            v_receptor,
            str(pdbqt_file),
            output_folder,
            f"docked_{lig_name}.pdbqt"
        )

        lig_idx = lig_name.split('-')[0]
        tautomer_idx = int(lig_name.split('-')[1])-1
        
        docking_result.append({
            'epoch': epoch,
            'name': lig_name,
            'docking_score': result.energies()[0,0],
            'smiles': [Chem.MolToSmiles(mol) for mol in Chem.SDMolSupplier(f'{prepared_ligands_path}/{lig_idx}.sdf') if mol][tautomer_idx]
            # Kinda convoluted, but no really good way of knowing which ligand has been prepared or not, so read it back from the sdf file that has been generated
        })
    return np.array(docking_result)

def vina_prepare_and_dock_ligand_first(
    starting_point,
    starting_point_name,
    prepare_ligand_script_path,
    v_receptor,
    prepared_ligands_path,
    output_folder
):
    # Docking info for the starting structure 
    prepare_ligand(starting_point_name, starting_point, prepare_ligand_script_path)
    lig_name = 'epoch_0-1'
    result = vina_dock_ligand(
                v_receptor,
                prepared_ligands_path+f'/{lig_name}.pdbqt', # Assuming there's only one isomer
                output_folder,
                f"docked_{lig_name}.pdbqt"
            )
    v_dict = {
        'epoch': 0,
        'name': lig_name,
        'docking_score': result.energies()[0,0],
        'smiles': starting_point
    }

    return v_dict

def sigmoid_acceptance_criteria(
    docking_result,
    prev_docking,
    a # Slope of sigmoid function // (lower it is, the more 'generous' it is in accepting lower docking score structures) 
):
    def sigmoid(x,a):
        return 1/(1 + np.exp(-a*x))

    accept_or_reject = []
    for i in docking_result:
        difference = prev_docking - i['docking_score']
        random = np.random.uniform()
        accept_or_reject.append(
            sigmoid(difference,1)>random
        )
    return accept_or_reject

In [3]:
# Clean direcotories
subprocess.run("rm prepared_ligands/*", shell=True);
subprocess.run("rm failed_ligands/*", shell=True);
subprocess.run("rm docked_poses/*", shell=True);
subprocess.run("rm *.pdbqt", shell=True);

rm: failed_ligands/*: No such file or directory


In [4]:
# Set parameters
n_epoch      = 100
n_rep        = 2
n_pert       = 50
SA_threshold = 5 # Setting it to 10 essentially removes the SA constraint
starting_point      = "CN(C([C@H](CC1=CC=CC=C1)NC(CC2=C(C)NC3=C2C=CC=C3)=O)=O)C4=CC=CC=C4" # PF74
starting_point_name = "epoch_0"

# Script paths 
prepare_ligand_script_path   = "script/prepare_ligand.sh"
prepare_receptor_script_path = "script/prepare_receptor.sh"

# Cluster population treshold
cluster_size_threshold = 4

# Folder name (without / at the end) of where the prepared ligands (.pdbqt) are saved
prepared_ligands_path = "prepared_ligands"

# Prepare receptor, which will not change throughout simulation 
receptor_name     = "4XFZ"
receptor_pdb_path = "reference/4xfz_two_chain_dry_clean.pdb"
prepare_receptor(
    receptor_name,
    receptor_pdb_path,
    prepare_receptor_script_path
)

# Prepare receptor grid to dock on 
receptor_pdbqt = "./4XFZ.pdbqt"
grid_center    = [19.61, -23.07, 0.3]
output_folder  = "docked_poses/"
v_receptor = vina_setup_receptor(receptor_pdbqt, grid_center)

/opt/anaconda3/envs/molpert/lib/python3.10/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
@> 4919 atoms and 1 coordinate set(s) were parsed in 0.02s.



Files written:
4XFZ.pdbqt <-- static (i.e., rigid) receptor input file
Computing Vina grid ... done.


In [ ]:
# Array to save the result 
# Will eventually turn into pd df
rows = []

# Docking info for the starting structure 
rows.append(
    vina_prepare_and_dock_ligand_first(
        starting_point,
        starting_point_name,
        prepare_ligand_script_path,
        v_receptor,
        prepared_ligands_path,
        output_folder
    )
)

# Need to set previous epoch's starting point's docking score 
prev_docking = rows[0]['docking_score']

# Propagate for epoch
for epoch in range(1,n_epoch):
    # Propagate the trajectory from the starting point 
    trajs = propagate_chemical_space_trajectory_reps(starting_point, n_rep, n_pert, SA_threshold)

    # Cluster the concatenated trajectory / get medoids (representative structure) 
    trajs_concat = np.concatenate(trajs)
    clusters, medoids = cluster_trajectory(trajs_concat, cluster_size_threshold)

    # Prepare ligands (medoid structures) 
    prepare_ligand_batch(
        medoids,
        trajs_concat,
        epoch,
        prepare_ligand_script_path
    )

    # Dock ligands to prepared receptor grid 
    docking_result = vina_dock_ligand_batch(
        prepared_ligands_path,
        epoch,
        v_receptor,
        output_folder
    )

    # Accept or reject and add the accepted structures to rows
    accept_or_reject = sigmoid_acceptance_criteria(
        docking_result,
        prev_docking,
        0.5 
    )

    # Determine what the next epoch's starting structure should be
    if docking_result[accept_or_reject].size == 0:
    # No structures were accepted / do nothing & keep the previous starting point / run again   
        continue
    else:
    # Accepted at least one structure
    # Out of N structure, select one at random and select it to be the next 
        # Append data to rows
        for i in docking_result[accept_or_reject]:
            rows.append(i)

        # Pick the starting structure for next epoch
        pick = np.random.choice(docking_result[accept_or_reject])
        pick_idx = int(pick['name'].split('_lig_')[1].split('-')[0])
        starting_point = Chem.MolToSmiles(trajs_concat[pick_idx])
        prev_docking   = pick['docking_score']

# Convert to dataframe
df = pd.DataFrame(rows, columns=['epoch', 'name', 'docking_score', 'smiles'])

Scrub completed.
Summary of what happened:
Input molecules supplied: 1
mols processed: 1, skipped by rdkit: 0, failed: 0
nr isomers (tautomers and acid/base conjugates): 1 (avg. 1.000 per mol)
nr conformers:  1 (avg. 1.000 per isomer, 1.000 per mol)


/opt/anaconda3/envs/molpert/lib/python3.10/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0
No duplicate molecule molecule names were found


/opt/anaconda3/envs/molpert/lib/python3.10/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


...


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -7.634          0          0
   2       -7.577       0.79      1.774
   3        -7.55      2.429      4.951
   4       -7.056      2.106      4.571
   5       -7.027       2.45      4.472
   6       -6.878      2.569      4.405
   7       -6.857      4.578      6.951
   8       -6.832      5.395      8.279
   9       -6.795      2.152      4.061
  10       -6.697      4.659      8.626
  11       -6.639      2.613      4.297
  12       -6.614      2.795      4.728
  13       -6.567      5.998      9.211
  14       -6.479      5.325      8.095
  15       -6.407      5.548      8.648
  16       -6.324       2.62      5.357
  17        -6.26      4.295      7.207
  18       -6.259      4.485       8.

7it [00:36,  5.60s/it]

02
  19         -4.4      2.337      4.829
  20       -4.375      2.455      4.785
Performing local search ... done.
Docking: epoch_2_lig_60-1
Performing docking (random seed: -236248410) ... 

8it [00:43,  6.16s/it]WARNING: At low exhaustiveness, it may be impossible to utilize all CPUs.
9it [00:51,  6.63s/it]


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -6.583          0          0
   2       -6.034      3.666      7.552
   3       -6.015      6.744      9.156
   4       -5.985      2.736      6.366
   5       -5.925      5.402      8.226
   6       -5.801      5.956      8.872
   7       -5.638      6.526      9.492
   8       -5.502      6.361      10.05
   9       -5.379      6.846      9.919
  10       -5.362      7.654      10.09
  11       -5.324      5.953       7.69
  12       -5.182      5.631      8.323
  13       -5.141      7.975      10.26
  14       -5.133       2.72      6.131
  15       -5.125      6.881      9.888
  16       -5.074      6.258       9.46
  17       -5.038      7.512      10.09
  18       -4.882      6.182      8.9

In [ ]:
df.to_csv(f'epoch_{n_epoch}_rep_{n_rep}_pert_{n_pert}.csv',index=False)